## Load Packages

In [155]:
# # Install tempo-embeddings from GitHub
# # This can also refer to a specific version or branch

%pip install -e ..

# %pip install --upgrade pip  # Required for properly resolving dependencies
# %pip uninstall -y tempo_embeddings  # Remove existing installation
# %pip install --upgrade git+https://github.com/Semantics-of-Sustainability/tempo-embeddings.git

Obtaining file:///Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for tempo_embeddings (pyproject.toml) ... done
  Created wheel for tempo_embeddings: filename=tempo_embeddings-0.0.1-0.editable-py3-none-any.whl size=9608 sha256=bf0cc30244d5d6db57eb7628b9a5e630b3ecda35e148947ce524b45d0c720222
  Stored in directory: /private/var/folders/d8/j5_fyf8941j_492zvf8948y40000gn/T/pip-ephem-wheel-cache-x3deq9wq/wheels/9d/dd/1c/d0e8ab6bde11a84556c00bd645333aaad0b90e2c38e11b0ed1
Successfully built tempo_embeddings
  Attempting uninstall: tempo_embeddings
    Found existing installation: tempo_embeddings 0.0.1
    Uninstalling tempo_embeddings-0.0.1:
      Successfully uninstalled tempo_embeddings-0.0.1
Note: you may need to restart the ke

In [156]:
# make sure installation has succeeded

import logging
from importlib import reload

reload(logging)
logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)

In [157]:
%load_ext autoreload
%autoreload now

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:
try:
    import google.colab  # noqa: F401

    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False

In [159]:
from pathlib import Path

from tempo_embeddings.text.corpus import Corpus

## Load Database Manager

The `db_path` parameter should point to the directory where the database is, so the original configuration and records are loaded. The database was created using the notebook `1_compute_embeddings_nl.ipynb`. If the given path does not exist, a new EMPTY database will be created there. 

A bigger `batch_size` could make the search faster but if it is too big you might run out of memory.

In [160]:
from ipywidgets import widgets

host_selector = widgets.RadioButtons(
    options=["local", "Research Cloud"],
    value="Research Cloud",
    description="Weaviate Database Host",
)
host_selector

RadioButtons(description='Weaviate Database Host', index=1, options=('local', 'Research Cloud'), value='Resear…

In [161]:
port_selector = widgets.IntText(value=8087, description="Weaviate Database Port")
port_selector

IntText(value=8087, description='Weaviate Database Port')

In [162]:
import weaviate
from tempo_embeddings.embeddings.model import SentenceTransformerModelWrapper
from tempo_embeddings.embeddings.weaviate_database import WeaviateDatabaseManager
from tempo_embeddings.settings import DEFAULT_LANGUAGE_MODEL

KNOWN_HOSTS = {"local": "localhost", "Research Cloud": "145.38.192.173"}

db = WeaviateDatabaseManager(
    client=weaviate.connect_to_local(
        host=KNOWN_HOSTS[host_selector.value], port=port_selector.value
    ),
    model=SentenceTransformerModelWrapper.from_pretrained(DEFAULT_LANGUAGE_MODEL),
)

03:56:54 INFO:HTTP Request: GET http://145.38.192.173:8087/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
03:56:54 INFO:HTTP Request: GET http://145.38.192.173:8087/v1/meta "HTTP/1.1 200 OK"
03:56:54 INFO:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
03:56:55 INFO:HTTP Request: GET http://145.38.192.173:8087/v1/schema/TempoEmbeddings "HTTP/1.1 200 OK"


## Choose from the available Collections in the Database

In [163]:
from ipywidgets import widgets

existing_colls = list(db.get_available_collections())
collection_selector = widgets.SelectMultiple(
    options=existing_colls,
    value=["ANP", "StatenGeneraal"],
    description="Choose a Collection:",
    disabled=False,
    style={"description_width": "initial"},
)

print("\nCollection Sizes")
print("----------------")
max_len = max(len(collection) for collection in existing_colls) + 1
for collection in existing_colls:
    print(f"{collection:{max_len}}\t{db.get_collection_count(collection)}")

collection_selector

03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"
03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"
03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"
03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"
03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"
03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"
03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"
03:56:55 INFO:HTTP Request: POST http://145.38.192.173:8087/v1/graphql "HTTP/1.1 200 OK"



Collection Sizes
----------------
AlgemeenDagblad 	1005229
ANP             	126466
Telegraaf       	631072
StatenGeneraal  	820215
Volkskrant      	781978
NRC             	884143
Trouw           	568898


SelectMultiple(description='Choose a Collection:', index=(1, 3), options=('AlgemeenDagblad', 'ANP', 'Telegraaf…

## Create Sub-Corpus

To make the processing and visualization easier, we will create a new `Corpus` comprising only a subet of the original Collection. This corpus will contain only the records of interest. This is done by querying the database with keyword and metadata constraints. In this example we allow to look for:

- **Filter Terms:** retrieve only passages that contain exactly the given keywords.
- **Year Range:** retrieve only the records which are inside the provided years
- **Neighbors:** This indicates how much to *expand* the search into more datapoints. The idea is to retrieve the *top_k* neighbors of the initially retrieved passages. Ideally this will give related passages that did not mention any of the keywords explicitly.

In [164]:
widget_year_range = widgets.IntRangeSlider(
    description="Year Range: ",
    min=1800,
    max=2020,
    step=1,
    value=(1950, 2000),
    style={"description_width": "initial"},
    layout=widgets.Layout(width="400px"),
)
widget_terms = widgets.Text(
    description="Filter Terms (comma separated)",
    value="duurzaam",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="600px"),
)
widget_neighbors = widgets.IntSlider(
    description="Expand Neighborhood Size: ",
    min=0,
    max=10,
    value=5,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="400px"),
)

### Display the Widgets to choose the parameters

In [165]:
display(widget_terms)
display(widget_year_range)

Text(value='duurzaam', description='Filter Terms (comma separated)', layout=Layout(width='600px'), style=TextS…

IntRangeSlider(value=(1950, 2000), description='Year Range: ', layout=Layout(width='400px'), max=2020, min=180…

### Execute the Search

No need to move the code manually here. All parameters are grabbed from the widget values

In [166]:
# Unpack values form Widget
year_from, year_to = widget_year_range.value
FILTER_TERMS = [s.strip() for s in widget_terms.value.split(",")]
# Execute Database Query
where_range = {"year_from": year_from, "year_to": year_to}
print(f"Searching terms {FILTER_TERMS} between year {year_from} and {year_to}")
corpus = sum(
    (
        db.get_corpus(
            collection,
            filter_words=FILTER_TERMS,
            where_obj=where_range,
            include_embeddings=True,
            limit=10000,
        )
        for collection in collection_selector.value
    ),
    start=Corpus([]),
)
print(f"Found {len(corpus)} items that match!")

Searching terms ['duurzaam'] between year 1950 and 2000
Found 9485 items that match!


## Cluster the Corpus

In [167]:
corpus.compress_embeddings()

array([[ 7.9572086,  6.7504673],
       [13.292913 ,  5.3754554],
       [12.267235 ,  6.1648526],
       ...,
       [10.599762 ,  9.885481 ],
       [12.629106 ,  9.127337 ],
       [ 9.957805 ,  4.161889 ]], dtype=float32)

### Choose Minimum Cluster Size

In [168]:
min_cluster_size_widget = widgets.IntSlider(
    value=25,
    min=5,
    max=len(corpus) // 10,
    step=1,
    description="Minimum Cluster Size:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
    style={"description_width": "initial"},
)

In [169]:
min_cluster_size_widget

IntSlider(value=25, continuous_update=False, description='Minimum Cluster Size:', max=948, min=5, style=Slider…

### Execute Clustering

In [170]:
# Arguments: min_cluster_size=10, cluster_selection_epsilon=0.1, ...
# See https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html for full list

# e.g. min_samples=10, cluster_selection_epsilon=0.2, cluster_selection_method="leaf"
clusters = corpus.cluster(
    min_cluster_size=min_cluster_size_widget.value, cluster_selection_epsilon=0.1
)
print(
    f"Found {len(clusters)} clusters in the corpus. (min cluster size is {min_cluster_size_widget.value})"
)

if len(clusters) > 100:
    raise ValueError(
        "Seems like you have too many clusters! Try with a bigger value for min_cluster_size to avoid memory issues"
    )

max_cluster_size = max(len(c.passages) for c in clusters)

Found 42 clusters in the corpus. (min cluster size is 25)


### Expand clusters

In [171]:
display(widget_neighbors)

IntSlider(value=5, description='Expand Neighborhood Size: ', layout=Layout(width='400px'), max=10, style=Slide…

In [172]:
from tqdm import tqdm

for cluster in tqdm(clusters, unit="cluster", desc="Expanding"):
    cluster.extend(db.neighbour_passages(cluster, k=widget_neighbors.value))

Expanding: 100%|██████████| 42/42 [01:07<00:00,  1.61s/cluster]


### Label the Clusters (TF-IDF)
#### Load Stopwords to avoid including them in the Cluster Labels

In [173]:
stopwords_file = Path("../tempo_embeddings/data/stopwords-filter-nl.txt")

with open(stopwords_file.absolute(), "rt") as f:
    stopwords = set(f.read().splitlines())

stopwords.update(
    {
        "wij",
        "we",
        "moeten",
        "heer",
        "mevrouw",
        "minister",
        "voorzitter",
        "gaat",
        "wel",
        "den",
    }
)

#### Iterate Clusters to assign labels and save a sample of Passages for inspection

In [174]:
import os

if not os.path.exists("clusters"):
    os.makedirs("clusters")

selected_metadata = ["year"]
cluster_sample = 100
centroid_based_sample = True

all_clusters_records, df_cluster_labels, df_cluster_meta = [], [], []
for cluster in clusters:
    cluster.set_topic_label(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=5)
    df = cluster.to_dataframe()
    centroid = cluster.centroid()
    label = cluster.label
    cluster_size = len(cluster.passages)
    # Compute Cluster Stats as a Subcorpus
    top_words = " ".join(
        cluster.top_words(exclude_words=frozenset(stopwords | set(FILTER_TERMS)), n=10)
    )
    all_clusters_records.append((f"{label}\t{cluster_size}\t{centroid}\t{top_words}\n"))
    df_cluster_labels.append(cluster.label)
    df_cluster_meta.append(df[selected_metadata])
    # Save a sample of the Cluster Passages in a File (the cluster_sample_size closest to the centroid)
    file_prefix = f"cluster_{year_from}_{year_to}_{cluster.label.replace('; ', '_')}"
    df_sample = cluster.to_dataframe(
        sample_size=cluster_sample, centroid_based_sample=centroid_based_sample
    )
    df_sample.to_csv(f"clusters/{file_prefix}.tsv", sep="\t", index=False)
    print(cluster)

/Users/carstenschnober/Documents/SemanticsOfSustainability/workspace/tempo-embeddings/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Subcorpus('concept; veilig; verkeer; verkeersveiligheid; vervoersysteem', [0, 254, 361, 502, 781, 2441, 2698, 2737, 3098, 3963])
Subcorpus('bestand; partijen; tussen; vergelijk; vrede', [1, 4, 11, 14, 15, 25, 43, 70, 72, 73])
Subcorpus('bezit; herstel; persoonlijk; regering; vorming', [2, 6, 7, 10, 13, 20, 22, 23, 24, 26])
Subcorpus('aanpak; bouw; bouwen; plan; project', [3, 55, 304, 458, 665, 1049, 1070, 1237, 1448, 1582])
Subcorpus('aanzien; arbeidsproces; goederen; ten; weigeringsgrond', [5, 1229, 1441, 1554, 1615, 1854, 1895, 1896, 1965, 2009])
Subcorpus('Outliers', [8, 9, 12, 17, 18, 19, 21, 29, 30, 33])
Subcorpus('europa; europese; multilateralisme; sociaal; weg', [16, 42, 249, 250, 251, 446, 448, 1090, 1147, 1618])
Subcorpus('begrip; begrippen; woord; «duurzaam»; „duurzaam', [27, 270, 461, 462, 464, 885, 1033, 1195, 1196, 1270])
Subcorpus('aardgas; duurzame; energie; gas; kernenergie', [36, 37, 38, 39, 322, 323, 1746, 1747, 2608, 2950])
Subcorpus('bosbeheer; bossen; geproduceerd

In [175]:
with open(f"clusters/clusters_all_{year_from}_{year_to}.csv", "wt") as f:
    f.write("Label\tSize\tCentroid\tTopWords\n")
    for rec in all_clusters_records:
        f.write(rec)

### Plot Cluster Content Distributions

In [176]:
import logging

import matplotlib.pyplot as plt
from ipywidgets import interact

logging.getLogger("matplotlib").setLevel(logging.WARNING)


# TODO: Maybe use something better than pyplot so the histogram is prettier and more flexible
def plot_cluster_distribution(cluster_index, column_name, num_bins):
    df = df_cluster_meta[cluster_index]
    cluster_name = df_cluster_labels[cluster_index]
    plt.figure(figsize=(10, 6))
    plt.hist(df[column_name], edgecolor="black", bins=num_bins)
    plt.xlabel(f"{column_name}", fontsize=14)
    plt.xticks(rotation=90)
    plt.ylabel("Frequency", fontsize=14)
    plt.title(f"{cluster_name}", fontsize=16)
    plt.show()


cluster_selector = widgets.Dropdown(
    options=[(lbl, i) for i, lbl in enumerate(df_cluster_labels)],
    description="Choose a Cluster:",
    disabled=False,
    style={"description_width": "initial"},
)

# Would be better to choose a BIN_SIZE: so it plots per year, per decade, per 25 years, etc...
hist_bins_selector = widgets.Dropdown(
    options=[x + 1 for x in range(100)],
    value=10,
    description="Choose Number of Bins:",
    disabled=False,
    style={"description_width": "initial"},
)

variable_selector = widgets.Dropdown(
    options=selected_metadata,
    description="Choose a Column to Plot:",
    disabled=False,
    style={"description_width": "initial"},
)

interact(
    plot_cluster_distribution,
    cluster_index=cluster_selector,
    column_name=variable_selector,
    num_bins=hist_bins_selector,
)

interactive(children=(Dropdown(description='Choose a Cluster:', options=(('concept; veilig; verkeer; verkeersv…

<function __main__.plot_cluster_distribution(cluster_index, column_name, num_bins)>

## Visualize Embeddings (All Clusters)

In [177]:
try:
    del visualizer
except NameError:
    pass

In [179]:
import os

from bokeh.io import output_notebook
from bokeh.plotting import show

from tempo_embeddings.visualization.bokeh import BokehInteractiveVisualizer

output_notebook()

meta_fields = corpus.metadata_fields()
meta_fields = ["year", "date", "issue", "provenance"]

visualizer = BokehInteractiveVisualizer(
    *clusters, metadata_fields=meta_fields, width=2500, height=500
)

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*"

show(visualizer.create_document)

Loading BokehJS ...

03:58:17 INFO:Starting Bokeh server version 3.3.2 (running on Tornado 6.3.2)
03:58:17 INFO:User authentication hooks NOT provided (default user enabled)
